In [2]:
import numpy as np
import itertools

### 1) Exact Inference

In [24]:
def compute_g(config, beta):
    x_t = config[:-1]; x_t1 = config[1:]
    power = beta * np.sum(x_t == x_t1)
    return power

def compute_f(config_t, config_t1, beta):
    power = beta * np.sum(config_t == config_t1)
    return power

def compute_muft(muft1, fs, gs):
    powers = (muft1 + gs).reshape(1, -1) + fs
    scale = np.max(powers, axis = 1).reshape(-1, 1)
    powers -= scale
    
    # Marginalise
    marg = np.sum(np.exp(powers), axis = 1).reshape(-1,1)
    marg_powers = np.log(marg) + scale
    
    return marg_powers.reshape(-1)

In [48]:
# 1. Beta = 0.01
beta = 0.01
fs = []; gs = []; combinations = []
for combi in itertools.product([0,1], repeat = 10):
    combinations.append(combi)
    row = []
    gs.append(compute_g(np.array(combi), beta))
    for combi2 in itertools.product([0,1], repeat = 10):
        row.append(compute_f(np.array(combi), np.array(combi2), beta))
    fs.append(row)

fs = np.array(fs); gs = np.array(gs); combinations = np.array(combinations)

In [49]:
muft = np.zeros(2**10)
for i in range(9):
    muft = compute_muft(muft, fs, gs)
    
marg_end = muft + gs

scale = np.max(marg_end, axis = 0)
marg_end -= scale
marg_prob = np.exp(marg_end) / np.sum(np.exp(marg_end))

results = np.zeros((2,2))
for i in range(results.shape[0]):
    for j in range(results.shape[1]):
        indexes = np.where((combinations[:, 0] == i) * (combinations[:, 9] == j))[0]
        results[i,j] = np.sum(marg_prob[indexes])
        
print(results)

[[0.25 0.25]
 [0.25 0.25]]


In [50]:
# 2. Beta = 1
beta = 1
fs = []; gs = []; combinations = []
for combi in itertools.product([0,1], repeat = 10):
    combinations.append(combi)
    row = []
    gs.append(compute_g(np.array(combi), beta))
    for combi2 in itertools.product([0,1], repeat = 10):
        row.append(compute_f(np.array(combi), np.array(combi2), beta))
    fs.append(row)

fs = np.array(fs); gs = np.array(gs); combinations = np.array(combinations)

In [51]:
muft = np.zeros(2**10)
for i in range(9):
    muft = compute_muft(muft, fs, gs)

marg_end = muft + gs

scale = np.max(marg_end, axis = 0)
marg_end -= scale
marg_prob = np.exp(marg_end) / np.sum(np.exp(marg_end))

results = np.zeros((2,2))
for i in range(results.shape[0]):
    for j in range(results.shape[1]):
        indexes = np.where((combinations[:, 0] == i) * (combinations[:, 9] == j))[0]
        results[i,j] = np.sum(marg_prob[indexes])
        
print(results)

[[0.28044728 0.21955272]
 [0.21955272 0.28044728]]


In [52]:
# 3. Beta = 4
beta = 4
fs = []; gs = []; combinations = []
for combi in itertools.product([0,1], repeat = 10):
    combinations.append(combi)
    row = []
    gs.append(compute_g(np.array(combi), beta))
    for combi2 in itertools.product([0,1], repeat = 10):
        row.append(compute_f(np.array(combi), np.array(combi2), beta))
    fs.append(row)

fs = np.array(fs); gs = np.array(gs); combinations = np.array(combinations)

In [53]:
muft = np.zeros(2**10)
for i in range(9):
    muft = compute_muft(muft, fs, gs)
    
marg_end = muft + gs

# Marginalise more stably
scale = np.max(marg_end, axis = 0)
marg_end -= scale
marg_prob = np.exp(marg_end) / np.sum(np.exp(marg_end))

results = np.zeros((2,2))
for i in range(results.shape[0]):
    for j in range(results.shape[1]):
        indexes = np.where((combinations[:, 0] == i) * (combinations[:, 9] == j))[0]
        results[i,j] = np.sum(marg_prob[indexes])
        
print(results)

[[4.99652024e-01 3.47975924e-04]
 [3.47975924e-04 4.99652024e-01]]


### 3) Gibbs Sampling

In [68]:
import random

In [54]:
def get_neighbors(grid_size, i_loc, j_loc):
    indexes = []
    if i_loc - 1 >= 0:
        indexes.append((i_loc - 1, j_loc))
    if i_loc + 1 < grid_size:
        indexes.append((i_loc + 1, j_loc))
    if j_loc - 1 >= 0:
        indexes.append((i_loc, j_loc - 1))
    if j_loc + 1 < grid_size:
        indexes.append((i_loc, j_loc + 1))
    return indexes

In [76]:
# Initialise beta = 0.01
lattice = np.random.choice([0,1], size = (10,10))
counts = np.zeros((2,2)); beta = 0.01

# iterate through the lattice, generate samples
node_order = [np.unravel_index(i,(10,10)) for i in range(10 * 10)]
for it in range(10000):
    random.shuffle(node_order)
    for n, (i,j) in enumerate(node_order):
        neigh_idxs = get_neighbors(10, i, j); count_same1 = 0; count_same0 = 0
        for idx in neigh_idxs:
            if lattice[idx] == 1:
                count_same1 += 1
            else:
                count_same0 += 1

        px1 = np.exp(beta * count_same1); px0 = np.exp(beta * count_same0)
        norm_px1 = px1 / (px1 + px0)
        lattice[i,j] = np.random.choice([0,1], p = [1 - norm_px1, norm_px1])

        # Add sample to counts
        r = lattice[0, 9]; c = lattice[9,9]
        counts[r,c] += 1
            
counts /= np.sum(counts)

print(counts)

[[0.244533 0.256924]
 [0.243636 0.254907]]


In [77]:
# Initialise beta = 4
lattice = np.random.choice([0,1], size = (10,10))
counts = np.zeros((2,2)); beta = 4

# iterate through the lattice, generate samples
node_order = [np.unravel_index(i,(10,10)) for i in range(10 * 10)]
for it in range(10000):
    random.shuffle(node_order)
    for n, (i,j) in enumerate(node_order):
        neigh_idxs = get_neighbors(10, i, j); count_same1 = 0; count_same0 = 0
        for idx in neigh_idxs:
            if lattice[idx] == 1:
                count_same1 += 1
            else:
                count_same0 += 1

        px1 = np.exp(beta * count_same1); px0 = np.exp(beta * count_same0)
        norm_px1 = px1 / (px1 + px0)
        lattice[i,j] = np.random.choice([0,1], p = [1 - norm_px1, norm_px1])

        # Add sample to counts
        r = lattice[0, 9]; c = lattice[9,9]
        counts[r,c] += 1
            
counts /= np.sum(counts)

print(counts)

[[7.46500e-03 2.37000e-04]
 [8.19000e-04 9.91479e-01]]


In [78]:
# Initialise beta = 1
lattice = np.random.choice([0,1], size = (10,10))
counts = np.zeros((2,2)); beta = 1

# iterate through the lattice, generate samples
node_order = [np.unravel_index(i,(10,10)) for i in range(10 * 10)]
for it in range(10000):
    random.shuffle(node_order)
    for n, (i,j) in enumerate(node_order):
        neigh_idxs = get_neighbors(10, i, j); count_same1 = 0; count_same0 = 0
        for idx in neigh_idxs:
            if lattice[idx] == 1:
                count_same1 += 1
            else:
                count_same0 += 1

        px1 = np.exp(beta * count_same1); px0 = np.exp(beta * count_same0)
        norm_px1 = px1 / (px1 + px0)
        lattice[i,j] = np.random.choice([0,1], p = [1 - norm_px1, norm_px1])

        # Add sample to counts
        r = lattice[0, 9]; c = lattice[9,9]
        counts[r,c] += 1
            
counts /= np.sum(counts)

print(counts)

[[0.286407 0.216441]
 [0.21627  0.280882]]


### 2) Variational Inference with MFA

In [89]:
def sigmoid(x):
    return 1 / (1 + np.exp (-x))

# init the q's randomly
qs = np.random.uniform(0,1, size = (10, 10))
beta = 0.01

# Iterate coordinate ascent
for it in range(10):
    for i in range(qs.shape[0]):
        for j in range(qs.shape[1]):
            neigh_idx = np.array(get_neighbors(10, i, j))
            q_neigh = qs[neigh_idx[:,0], neigh_idx[:, 1]]
            r = np.sum(2 * q_neigh - 1) * beta
            qs[i,j] = sigmoid(r)

results = np.zeros((2,2)); q1 = qs[0,9]; q2 = qs[9,9]
for i in range(results.shape[0]):
    for j in range(results.shape[1]):
        qi = q1 if i == 1 else (1 - q1); qj = q2 if j == 1 else (1 - q2)
        results[i,j] = qi * qj
print(results)

[[0.25 0.25]
 [0.25 0.25]]


In [90]:
def sigmoid(x):
    return 1 / (1 + np.exp (-x))

# init the q's randomly
qs = np.random.uniform(0,1, size = (10, 10))
beta = 1

# Iterate coordinate ascent
for it in range(10):
    for i in range(qs.shape[0]):
        for j in range(qs.shape[1]):
            neigh_idx = np.array(get_neighbors(10, i, j))
            q_neigh = qs[neigh_idx[:,0], neigh_idx[:, 1]]
            r = np.sum(2 * q_neigh - 1) * beta
            qs[i,j] = sigmoid(r)

results = np.zeros((2,2)); q1 = qs[0,9]; q2 = qs[9,9]
for i in range(results.shape[0]):
    for j in range(results.shape[1]):
        qi = q1 if i == 1 else (1 - q1); qj = q2 if j == 1 else (1 - q2)
        results[i,j] = qi * qj
print(results)

[[0.02416018 0.13210702]
 [0.13044795 0.71328485]]


In [106]:
def sigmoid(x):
    return 1 / (1 + np.exp (-x))

# init the q's randomly
qs = np.random.uniform(0,1, size = (10, 10))
beta = 4

# Iterate coordinate ascent
for it in range(10):
    for i in range(qs.shape[0]):
        for j in range(qs.shape[1]):
            neigh_idx = np.array(get_neighbors(10, i, j))
            q_neigh = qs[neigh_idx[:,0], neigh_idx[:, 1]]
            r = np.sum(2 * q_neigh - 1) * beta
            qs[i,j] = sigmoid(r)

results = np.zeros((2,2)); q1 = qs[0,9]; q2 = qs[9,9]
for i in range(results.shape[0]):
    for j in range(results.shape[1]):
        qi = q1 if i == 1 else (1 - q1); qj = q2 if j == 1 else (1 - q2)
        results[i,j] = qi * qj
print(results)

[[1.12481876e-07 3.35270697e-04]
 [3.35270697e-04 9.99329346e-01]]
